In [8]:
import nutils as ntl
import numpy as np
import matplotlib.pyplot as plt
import inspect


## SUPG

We use SUPG(iNsErT mEtHoD hErE) to solve the advection dominated skew to mesh benchmark problem as described in [links go here]:

$$-\kappa \Delta u + \beta\cdot \nabla u = 0  $$

with $\kappa = 10^{-3}$, $\beta = (.5, 1)$ on a square grid with dirichlet boundary condition 

$$u = \begin{cases} 1 & y = 0 \text{ or } x = 0 \text{ and } y < 0.2 \\ 0 & \text{otherwise} \end{cases}   $$ 

In [9]:
degree = 1 #degree of basis functions
nelems = 9 #number of elements along edge of unit square
etype = 'square'

topo, geom =  ntl.mesh.unitsquare(nelems, etype) #rectilinear([np.linspace(0,1, nx), np.linspace(0,1, ny) ])
ns = ntl.function.Namespace()
ns.x = geom
ns.beta= np.array([0.5, 1.0])
ns.kappa = 1e-3
ns.basis = topo.basis('lagrange', degree = degree)
ns.u = 'basis_i ?dofs_i'

#element intrinsic time scale
ns.h = 0.5*np.sqrt(2/nelems**2) #mesh size parameter
ns.betanorm = 'beta_i beta_i'
ns.tau = 'h / (2 betanorm)'

#dirichlet boundary condition

#residual
res = topo.integral( 'kappa basis_i,j u_,j d:x' @ ns, degree = 2)
res += topo.integral( 'tau beta_k basis_i,k (-kappa u_,jj + beta_n u_,n)  d:x' @ ns, degree = 2) 

In [10]:
#print((beta *beta).sum(0)) #ct.getfullargspec(topo.basis))
#elp(np.dot)#(topo.basis)
help(topo.boundary.indicator)

Help on method indicator in module nutils.topology:

indicator(subtopo) method of nutils.topology.UnionTopology instance

